#1 - Introduction

This notebook presents the Teaching-Learning-Based Optimization (TLBO) algorithm, a metaheuristic optimization technique inspired by the teaching process in a classroom.

#2 - Setup and Imports

### 2.1 - import the necessary libraries

In [1]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

### 2.2 - Define problem parameters

In [2]:
# Define problem parameters
lower_bound = -5  # Lower bound of the search space.
upper_bound = 5   # Upper bound of the search space.

num_variables = 4    # Number of decision variables.
population_size = 5  # Size of the population.

TF = 2  # Teaching factor

num_iterations = 1000      # Number of iterations.
threshold_fitness = 0.01  # Threshold fitness value.

### 2.3 - Optimization Function Definition


In [3]:
# Define the optimization function
def optimization_function(x1, x2, x3, x4):
    """Objective function."""
    return x1**2 + x2**2 + x3**2 + x4**2

#3 - TLBO Utility Functions


### 3.1 - Initialization Function

In [4]:
def initialize_population(population_size, lower_bound, upper_bound, num_variables):
    population = []
    for _ in range(population_size):
        individual = np.random.uniform(lower_bound, upper_bound, num_variables)  # Generating a random individual within the specified bounds.
        population.append(individual)
    return population

### 3.2 - Teaching Phase

In [5]:
def teaching_phase(population, fitness_values):
    best_index = np.argmin(fitness_values)    # Finding the index of the individual with the best fitness.
    teacher = population[best_index]          # Selecting the best individual as the teacher.
    mean_value = np.mean(population, axis=0)  # Calculating the mean of all individuals in the population.

    # Performing teaching phase operation.
    for i, individual in enumerate(population):
        ri = np.random.uniform(0, 1, size=len(teacher))
        new_individual = individual + ri * (teacher - TF * mean_value)

        # Boundary check
        new_individual = np.clip(new_individual, lower_bound, upper_bound)

        # Updating individual in population with new value.
        population[i] = new_individual

     # Calculating fitness values for the updated population.
    fitness_values = [optimization_function(*individual) for individual in population]

    return population

### 3.3 - Learning Phase

In [6]:
def learning_phase(population, fitness_values):
    partner_indices = np.random.choice(len(population), size=len(population), replace=True)

    for i, individual in enumerate(population):
        partner_index = partner_indices[i]
        partner = population[partner_index]
        ri = np.random.uniform(0, 1, size=len(individual))

        if fitness_values[i] < fitness_values[partner_index]:
            new_individual = individual + ri * (partner - individual)  # Performing learning phase operation if individual is better.
        else:
            new_individual = individual - ri * (partner - individual)  # Performing learning phase operation if partner is better.

        # Boundary check
        new_individual = np.clip(new_individual, lower_bound, upper_bound)
        # Updating individual in population with new value.
        population[i] = new_individual

    # Calculating fitness values for the updated population.
    fitness_values = [optimization_function(*individual) for individual in population]

    return population

### 3.4 - Population points

In [7]:
def print_population_info(population, fitness_values, iteration=None):
    # Iterating through each individual in the population and printing their rounded values and fitness.
    for i, individual in enumerate(population):
        rounded_ind = np.round(individual, 2)
        print(f"Individual {i+1}: {rounded_ind}\t Fitness: {fitness_values[i]:.2f}")

### 3.5 - Final Best point

In [8]:
def final_result(fitness_values, population):
    # Finding the minimum fitness value in the fitness_values list.
    min_fitness = min(fitness_values)
    # Finding the index of the minimum fitness value in the fitness_values list.
    min_index = fitness_values.index(min_fitness)
    # Retrieving the solution corresponding to the minimum fitness value.
    min_solution = population[min_index]
    # Rounding the minimum solution values for neatness.
    rounded_min_solution = np.round(min_solution, 2)
    # Printing the minimum solution and its fitness value.
    print(f"\nMinimum Solution: {rounded_min_solution}\t Fitness: {min_fitness:.2f}")

    # Printing a separator.
    print("\n","=" * 65)

# 4 - Iteration Function


In [9]:
def tlbo_iteration(population, fitness_values, iteration):
    print(f"\nIteration {iteration}:")
    print('-'*15)

    # Printing information about each individual in the population.
    print_population_info(population, fitness_values)

    print('-'*65)
    teacher_index = np.argmin(fitness_values)
    teacher_fitness = fitness_values[teacher_index]
    rounded_teacher_index = np.round(population[teacher_index], 2)
    print(f"Teacher: {rounded_teacher_index}\t Fitness: {teacher_fitness:.2f}")

    print('-'*65)
    population_mean = np.mean(population, axis=0)
    rounded_mean_value = np.round(population_mean, 2)
    print(f"Population Mean: {rounded_mean_value}")

    print('-'*65)
    population = teaching_phase(population, fitness_values)
    print("[1] The Population After Teaching Phase:")
    print('-'*40)
    print_population_info(population, fitness_values)

    print('-'*65)
    partner_index = np.random.randint(len(population))
    rounded_partner_index = np.round(population[partner_index], 2)
    print(f"Partner: {rounded_partner_index}")

    print('-'*65)
    population = learning_phase(population, fitness_values)
    print("[2] The Population After Learning Phase:")
    print('-'*40)
    print_population_info(population, fitness_values)
    print("\n","=" * 65)

    return population

#5 - Main Function


In [10]:
def main():
    # Initialize population
    population = initialize_population(population_size, lower_bound, upper_bound, num_variables)

    # Track fitness values for each individual in the population
    fitness_values = [optimization_function(*individual) for individual in population]

    # Track best solution and its fitness value
    best_fitness = min(fitness_values)
    best_solution = population[np.argmin(fitness_values)]

    # Track convergence
    convergence = [best_fitness]

    # Perform TLBO iterations with a maximum limit and convergence criterion
    iteration = 0
    while iteration < num_iterations:  # Maximum number of iterations
        # Teaching and Learning phases
        population = tlbo_iteration(population, fitness_values, iteration + 1)

        # Update fitness values
        fitness_values = [optimization_function(*individual) for individual in population]

        # Check convergence criterion
        new_best_fitness = min(fitness_values)
        if abs(new_best_fitness - best_fitness) < threshold_fitness:
            break  # Exit loop if convergence criterion is met
        else:
            best_fitness = new_best_fitness
            convergence.append(best_fitness)

        iteration += 1

    # Print final results
    final_result(fitness_values, population)

In [11]:
if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
Individual 4: [-2.48 -2.91  0.24 -4.47]	 Fitness: 28.97
Individual 5: [-0.22  1.12  3.66  1.98]	 Fitness: 49.92


Iteration 778:
---------------
Individual 1: [-2.53 -3.4  -5.   -5.  ]	 Fitness: 67.95
Individual 2: [ 5.    5.    2.27 -4.01]	 Fitness: 71.19
Individual 3: [-3.08  5.   -3.97 -2.58]	 Fitness: 56.87
Individual 4: [-2.48 -2.91  0.24 -4.47]	 Fitness: 34.68
Individual 5: [-0.22  1.12  3.66  1.98]	 Fitness: 18.65
-----------------------------------------------------------------
Teacher: [-0.22  1.12  3.66  1.98]	 Fitness: 18.65
-----------------------------------------------------------------
Population Mean: [-0.66  0.96 -0.56 -2.82]
-----------------------------------------------------------------
[1] The Population After Teaching Phase:
----------------------------------------
Individual 1: [-2.48 -3.55 -2.72 -1.88]	 Fitness: 67.95
Individual 2: [ 5.    4.53  4.14 -1.79]	 Fitness: 71.19
Individual 3: [-2.32  4.24 -3.21 -1.88

#6 - Conclusion

The Teaching-Learning-Based Optimization (TLBO) algorithm presents a promising approach to optimization tasks. Its simplicity and effectiveness make it suitable for various problem domains. While further research may enhance its performance, TLBO stands as a valuable tool for tackling optimization challenges.